In [ ]:
from auxiliar import *
import torch
import librosa

def frft_angle_manipulation(signal, angles, windows_size, hop_length, complex_part="real_part", window_type_input="rectangular", window_type_output="hann",fs=44100):
    for windows_size, hop_length in zip(windows_size, hop_length):
        output = frft_windowed(signal, windows_size, hop_length, angles, complex_part, window_type_input, window_type_output)
        print(f"Angle: {angle}, Window size: {windows_size}, Hop length: {hop_length}")
        display(Audio(output, rate=fs))
    
angles = [0.01, 0.05, 0.1]
windows_size, hop_length = [2048, 2048*2, 2048*4], [1024, 1024*2, 1024*4]

#read with librosa
# for files in folder path
for file in os.listdir("../sounds/"):
    if file.endswith(".wav"):
        print(f"Processing {file}")
        signal = librosa.load(os.path.join("../sounds/", file), sr=44100, mono=True)[0]
        signal = torch.tensor(signal)
        signal = signal[:44100*5]  # 5 seconds of audio
        print(f"Original Signal")
        display(Audio(signal.cpu().numpy(), rate=44100))
        for angle in angles:
            frft_angle_manipulation(signal, angle, windows_size, hop_length, complex_part="real_part", window_type_input="rectangular", window_type_output="hann", fs=44100)